**นำข้อมูลข้อง Zillow Price มาเพื่อทำ EDA**

นำเข้า Library ที่เกี่ยวข้อง

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

นำเข้าข้อมูลจาก Zillow Price 


In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

นำเข้าข้อมูลสำหรับการ Train -

transaction date คือ วันที่ถูกซื้อขายจึงต้องนำเข้าในรูปแบบวันที่

In [ ]:
train_df = pd.read_csv("../input/train_2016_v2.csv", parse_dates=["transactiondate"])
train_df.shape

In [ ]:
train_df.head()

**Logerror:**

เนื่องจากเป้าหมายของ dataset นี้นั้นใช้ logerror ในการทำผลลัพธ์ จึงควรลองตรวจสอบดูก่อน 


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(range(train_df.shape[0]), np.sort(train_df.logerror.values))
plt.xlabel('index', fontsize=12)
plt.ylabel('logerror', fontsize=12)
plt.show()

ดูดีเลยสำหรับ outliner เรามาลองเอา outliner ออกแล้วทำ Histogram

**Transaction Date:**

ลองดูวิธีการการจายตัวของ Transaction ในแต่ละเดือน

In [ ]:
train_df['transaction_month'] = train_df['transactiondate'].dt.month

cnt_srs = train_df['transaction_month'].value_counts()
plt.figure(figsize=(12,6))
sns.barplot(cnt_srs.index, cnt_srs.values, alpha=0.8, color=color[3])
plt.xticks(rotation='vertical')
plt.xlabel('Month of transaction', fontsize=12)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.show()

ลองมาดูว่าใน dataset นี้มีหลังไหนที่มีการซื้อขายมากว่า 1 ครั้ง 

In [ ]:
(train_df['parcelid'].value_counts().reset_index())['parcelid'].value_counts()

In [ ]:
sum = 90026+123+1
sum

In [ ]:
def percentage(part, whole):
  percentage = 100 * float(part)/float(whole)
  return str(percentage) + "%"

print(percentage(123, sum))

ซึ่งใน Dataset นี้ข้อมูลส่วนมากจะมีประวัติการซื้อขายเพียงครั้งเดียวเท่านั้น 
ข้อมูลของประกาศที่การซื้อขายสองครั้งคือ 123 ครั้งหรือ13%
และประกาศที่มีการซื้อขาย 3 ครั้งมีเพียงข้อมูลเดียว

**Properties 2016:**

คราวนี้มาลองดูข้อมูล Properties 2016 

In [ ]:
prop_df = pd.read_csv("../input/properties_2016.csv")
prop_df.shape

In [ ]:
prop_df.info()

In [ ]:
prop_df.head()

In [ ]:
prop_df.isna().sum()

หลังจากที่ดูคร่าวๆ คิดว่าน่าจะมีค่าว่าง
ลองมาดูว่าในแต่ละตัวแปรมีค่าว่างมากน้อยแค่ไหน

In [ ]:
missing_df = prop_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_count.values, color='blue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show()

ลองเอาข้อมูลละติจูดและลองติจูมา plot ดูกันนะครับ 

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=prop_df.latitude.values, y=prop_df.longitude.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()


จากข้อมูลที่เรามีพบว่า ประกาศที่เรามีนั้นเป็นประกาศที่อยู่ในรัฐ Los Angeles, Orange Ventura, California

เรามีข้อมูลใน Train 90,811 ประกาศ และเรามีข้อมูลทั้งหมดจำนวน 2,985,217 ประกาศ 

ถึงเวลาที่เราจะรวมไฟล์ทั้งสองเข้ามาด้วยกันเพื่อทำการวิเคราะห์ต่อไป 

In [ ]:
train_df = pd.merge(train_df, prop_df, on='parcelid', how='left')
train_df.head()

ตรวจสอบ Data Type ว่าเป็นยังไงบ้าง

In [ ]:
pd.options.display.max_rows = 65

dtype_df = train_df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
dtype_df

Almost all are float variables with few object (categorical) variables. Let us get the count. 

In [ ]:
dtype_df.groupby("Column Type").aggregate('count').reset_index()

ตรวจสอบ Missing Value หลังจากที่ได้ทำได้รวม 2 dataset เข้าด้วยกัน

In [ ]:
missing_df = train_df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df['missing_ratio'] = missing_df['missing_count'] / train_df.shape[0]
missing_df.loc[missing_df['missing_ratio']>0.999]

เราพบว่ามีอยู่ 4 ตัวแปรที่ข้อมูลกว่า 99% นั้นไม่เป็น Missing Value

**Univariate Analysis:**

Since there are so many variables, let us first take the 'float' variables alone and then get the correlation with the target variable to see how they are related.

In [ ]:
# Let us just impute the missing values with mean values to compute correlation coefficients #
mean_values = train_df.mean(axis=0)
train_df_new = train_df.fillna(mean_values, inplace=True)

# Now let us look at the correlation coefficient of each of these variables #
x_cols = [col for col in train_df.columns if col not in ['logerror'] if train_df[col].dtype=='float64']

labels = []
values = []
for col in x_cols:
    labels.append(col)
    values.append(np.corrcoef(train_df[col].values, train_df.logerror.values)[0,1])
corr_df = pd.DataFrame({'col_labels':labels, 'corr_values':values})
corr_df = corr_df.sort_values(by='corr_values')
    
ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(12,40))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='y')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.col_labels.values, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficient of the variables")
#autolabel(rects)
plt.show()

ความสัมพันธ์ระหว่างตัวแปรต้นและตัวแปรอิสระนั้นโดยรวมค่อยข้างต่ำ 

และมีบางตัวแปรที่ไม่มีความสัมพัธ์ใดเลย อาจจะเป็นเพราะเป็นเพียงค่าที่ไม่เกี่ยวข้องกับอะไรเลย


In [ ]:
corr_zero_cols = ['assessmentyear', 'storytypeid', 'pooltypeid2', 'pooltypeid7', 'pooltypeid10', 'poolcnt', 'decktypeid', 'buildingclasstypeid']
for col in corr_zero_cols:
    print(col, len(train_df[col].unique()))

เราลองมาทดสอบตัวแปรที่มีความสัมพันธ์ดูนะครับ

In [ ]:
corr_df_sel = corr_df.loc[(corr_df['corr_values']>0.02) | (corr_df['corr_values'] < -0.01)]
corr_df_sel

In [ ]:
cols_to_use = corr_df_sel.col_labels.tolist()

temp_df = train_df[cols_to_use]
corrmat = temp_df.corr(method='spearman')
f, ax = plt.subplots(figsize=(8, 8))

# Draw the heatmap using seaborn
sns.heatmap(corrmat, vmax=1., square=True)
plt.title("Important variables correlation map", fontsize=15)
plt.show()

The important variables themselves are very highly correlated.! Let us now look at each of them.
ตัวแปรที่สำคัญนั้นมีความ correlated กันค่อนข้างสูง 

เราลองวิเคราะห์ในแต่ละตัวแปรดูนะครับ

**Finished SquareFeet 12:**

Let us seee how the finished square feet 12 varies with the log error.

In [ ]:
col = "finishedsquarefeet12"
ulimit = np.percentile(train_df[col].values, 99.5)
llimit = np.percentile(train_df[col].values, 0.5)
train_df[col].loc[train_df[col]>ulimit] = ulimit
train_df[col].loc[train_df[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=train_df.finishedsquarefeet12.values, y=train_df.logerror.values, size=10, color=color[2])
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Finished Square Feet 12', fontsize=12)
plt.title("Finished square feet 12 Vs Log error", fontsize=15)
plt.show()

**คำนวน finished square feet:**

In [ ]:
col = "calculatedfinishedsquarefeet"
ulimit = np.percentile(train_df[col].values, 99.5)
llimit = np.percentile(train_df[col].values, 0.5)
train_df[col].loc[train_df[col]>ulimit] = ulimit
train_df[col].loc[train_df[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=train_df.calculatedfinishedsquarefeet.values, y=train_df.logerror.values, size=10, color=color[5])
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Calculated finished square feet', fontsize=12)
plt.title("Calculated finished square feet Vs Log error", fontsize=15)
plt.show()


**นับจำนวนห้องน้ำ:**

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="bathroomcnt", data=train_df)
plt.ylabel('Count', fontsize=12)
plt.xlabel('Bathroom', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of Bathroom count", fontsize=15)
plt.show()

จำนวนห้องน้ำประเภท 2 ห้องน้ำแยอะที่สุดและมีบางหลังที่ไม่มีห้องน้ำในตัวด้วย

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x="bathroomcnt", y="logerror", data=train_df)
plt.ylabel('Log error', fontsize=12)
plt.xlabel('Bathroom Count', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("How log error changes with bathroom count?", fontsize=15)
plt.show()

**นับจำนวนห้องนอน:**

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x="bedroomcnt", data=train_df)
plt.ylabel('Frequency', fontsize=12)
plt.xlabel('Bedroom Count', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Frequency of Bedroom count", fontsize=15)
plt.show()

จำนวนห้องนอนที่แยอะที่สุดคือ 3 ห้องนอน

In [ ]:
train_df['bedroomcnt'].loc[train_df['bedroomcnt']>7] = 7
plt.figure(figsize=(12,8))
sns.violinplot(x='bedroomcnt', y='logerror', data=train_df)
plt.xlabel('Bedroom count', fontsize=12)
plt.ylabel('Log Error', fontsize=12)
plt.show()

In [ ]:
col = "taxamount"
ulimit = np.percentile(train_df[col].values, 99.5)
llimit = np.percentile(train_df[col].values, 0.5)
train_df[col].loc[train_df[col]>ulimit] = ulimit
train_df[col].loc[train_df[col]<llimit] = llimit

plt.figure(figsize=(12,12))
sns.jointplot(x=train_df['taxamount'].values, y=train_df['logerror'].values, size=10, color='g')
plt.ylabel('Log Error', fontsize=12)
plt.xlabel('Tax Amount', fontsize=12)
plt.title("Tax Amount Vs Log error", fontsize=15)
plt.show()

**ที่สร้าง:**


In [ ]:
from ggplot import *
ggplot(aes(x='yearbuilt', y='logerror'), data=train_df) + \
    geom_point(color='steelblue', size=1) + \
    stat_smooth()

ลองมาดูละติจูดและลองติจูด

In [ ]:
ggplot(aes(x='latitude', y='longitude', color='logerror'), data=train_df) + \
    geom_point() + \
    scale_color_gradient(low = 'red', high = 'blue')

In [ ]:
train_y = train_df['logerror'].values
cat_cols = ["hashottuborspa", "propertycountylandusecode", "propertyzoningdesc", "fireplaceflag", "taxdelinquencyflag"]
train_df = train_df.drop(['parcelid', 'logerror', 'transactiondate', 'transaction_month']+cat_cols, axis=1)
feat_names = train_df.columns.values

from sklearn import ensemble
model = ensemble.ExtraTreesRegressor(n_estimators=25, max_depth=30, max_features=0.3, n_jobs=-1, random_state=0)
model.fit(train_df, train_y)

## plot the importances ##
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
indices = np.argsort(importances)[::-1][:20]

plt.figure(figsize=(12,12))
plt.title("Feature importances")
plt.bar(range(len(indices)), importances[indices], color="r", yerr=std[indices], align="center")
plt.xticks(range(len(indices)), feat_names[indices], rotation='vertical')
plt.xlim([-1, len(indices)])
plt.show()

Seems "tax amount" is the most importanct variable followed by "structure tax value dollar count" and "land tax value dollor count"



In [ ]:
from sklearn.preprocessing import StandardScaler , RobustScaler
from sklearn.model_selection import train_test_split , cross_val_predict , RandomizedSearchCV
from sklearn.linear_model import Ridge , Lasso , ElasticNet
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor , ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn import linear_model


In [ ]:
train_df , x_validation , train_y , y_validation = train_test_split(train_df , train_y , test_size=0.15 , shuffle=True , random_state=55)

In [ ]:
lis_model_score = []

def modeling(model) :
    pipe = Pipeline([
        ('scaling' , StandardScaler()) , 
        ('model' , model)
    ])
    
    prediction = cross_val_predict(pipe , train_df , train_y , cv=5)
    lis_model_score.append([
        model ,
        r2_score(prediction , train_y).round(4) , 
        mean_absolute_error(prediction , train_y).round(2) ,
        np.sqrt(mean_squared_error(prediction , train_y)).round(2)
    ])

In [ ]:
lis_algorithm =  [Lasso() , ElasticNet()]

In [ ]:
for model in lis_algorithm :
    modeling(model)

In [ ]:
result = pd.DataFrame(lis_model_score , columns=['Model' , 'R2 score' , 'MAE score' , 'RMSE score']).sort_values('MAE score')
result.head(2)